In [4]:
import pandas as pd
import numpy as np
import seaborn as sns

from os.path import join

In [5]:
PROJ_DIR = "/Volumes/projects_herting/LABDOCS/Personnel/Katie/deltaABCD_clustering/"
DATA_DIR = "data/"
FIGS_DIR = "figures/"
OUTP_DIR = "output/"

In [6]:
df = pd.read_csv(join(PROJ_DIR, DATA_DIR, "data.csv"), index_col=0, header=0)
df.drop(list(df.filter(regex='lesion.*').columns), axis=1, inplace=True)
no_2yfu = df[df["interview_date.2_year_follow_up_y_arm_1"].isna() == True].index
df = df.drop(no_2yfu, axis=0)

/Users/katherine.b/Library/Python/3.8/lib/python/site-packages/IPython/core/interactiveshell.py:3457: DtypeWarning: Columns (6295,6296,6297,6298,6299,6300,6311,6312,6401,6402,6403,6404,6405,6406,6417,6418,7656,7657,7658,7659,7660,7661,7674,7675) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
qc_vars = ["imgincl_dmri_include",
           "imgincl_rsfmri_include",
           "rsfmri_c_ngd_meanmotion",
           "rsfmri_c_ngd_ntpoints",
           "imgincl_t1w_include",
           "mrif_score"]

In [17]:
keep = df['mrif_score.baseline_year_1_arm_1'].between(1,2).index

In [21]:
# modality-specific filtering via masks
rsfmri_mask1 = df['imgincl_rsfmri_include'] == 1
rsfmri_mask2 = df['rsfmri_c_ngd_ntpoints'] >= 750.
rsfmri_mask = rsfmri_mask1 * rsfmri_mask2

smri_mask = df['imgincl_t1w_include'] == 1

dmri_mask = df['imgincl_dmri_include'] == 1

KeyError: 'imgincl_rsfmri_include'